In [8]:
import os
import torch
import torch.nn as nn # Neural Network
import torch.optim as optim # Optimizer's Relu etc. 
from torch.utils.data import Dataset, DataLoader # Preprocessing Dataset, Loading Dataset

from PIL import Image # Open Image
import torchvision.transforms as transforms # Image Processing & Augmentation i.e. Resize, Flip, Normalize etc. 
    
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity # Measures Similarity Between Vectors
from sklearn.preprocessing import MultiLabelBinarizer

# Device Configuration

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Loading the Dataset

In [14]:
import pandas as pd

movies = pd.read_csv('/kaggle/input/movie-recommendation-system/movies.csv')

ratings = pd.read_csv('/kaggle/input/movie-recommendation-system/ratings.csv')

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Genre Preprocessing

## Convert genre string → list

In [11]:
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))

## Multi-Hot Encode genres

In [12]:
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movies['genres'])

genre_matrix.shape

(62423, 20)

# Pytorch Dataset

In [13]:
class MovieDataset(Dataset):
    def __init__(self, genre_matrix):
        self.X = torch.tensor(genre_matrix, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.X[idx]  # autoencoder-style